In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import logging
import os
import torch
import webdataset as wds

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
def sentinel(f):
    return f if f is not None else 0.0

In [5]:
tar_file = 'data/mini_pieman.tar'
mask_file = 'data/pieman_data/wholebrain.nii.gz'

existed = os.path.isfile(tar_file)
sink = wds.TarWriter(tar_file)

OVERWRITE = True

if not existed or OVERWRITE:
    data_files = utils.sorted_glob('data/pieman_data/*.nii')
    
    total_trs = 0
    metadata = {
        'blocks': []
    }
    for i, data_file in enumerate(data_files):
        segments = os.path.splitext(os.path.basename(data_file))[0].split('-')
        subject = int(segments[1])
        task = segments[3]
        block = niidb.FMriActivationBlock(zscore=False)
        block.block = i
        block.filename = data_file
        block.mask = mask_file
        block.subject = subject
        block.task = task
        block.load()

        metadata['blocks'].append(block.wds_metadata())

        for vals in block.format_wds():
            sink.write(vals)
        block_trs = (block.end_time - block.start_time)
        total_trs += block_trs

        logging.info('Block %d %s of run %d started at %f, ended at %f, contained %d TRs', i,
                     block.task, block.run, sentinel(block.start_time), sentinel(block.end_time),
                     block_trs)
    
    metadata['voxel_locations'] = block.locations
    metadata['num_times'] = total_trs
    torch.save(metadata, tar_file + '.meta')
    logging.info('Recorded metadata, including voxel locations')

sink.close()

06/09/2021 11:19:41 Loading Nifti image data/pieman_data/sub-001-task-intact1.nii with mask data/pieman_data/wholebrain.nii.gz (zscore=False, smooth=None, zscore_by_rest=False)
06/09/2021 11:19:43 Block 0 intact1 of run 0 started at 0.000000, ended at 300.000000, contained 300 TRs
06/09/2021 11:19:43 Loading Nifti image data/pieman_data/sub-002-task-intact1.nii with mask data/pieman_data/wholebrain.nii.gz (zscore=False, smooth=None, zscore_by_rest=False)
06/09/2021 11:19:45 Block 1 intact1 of run 0 started at 0.000000, ended at 300.000000, contained 300 TRs
06/09/2021 11:19:45 Loading Nifti image data/pieman_data/sub-003-task-intact1.nii with mask data/pieman_data/wholebrain.nii.gz (zscore=False, smooth=None, zscore_by_rest=False)
06/09/2021 11:19:48 Block 2 intact1 of run 0 started at 0.000000, ended at 300.000000, contained 300 TRs
06/09/2021 11:19:48 Recorded metadata, including voxel locations
